# Metehan Ayhan - Recommender system for a medical supplies company project

Bu proje, bir medikal malzeme şirketinin satışlarını artırmak amacıyla öneri sistemi geliştirmeyi hedeflemektedir. Proje kapsamında, eksik verilerin analiz edilmesi, en popüler ürünlerin satış hacmi ve dolar tutarı bazında belirlenmesi, en çok satın alma yapan firmanın bulunması ve iki farklı öneri sistemi oluşturulması planlanmaktadır. İlk olarak, popülerlik tabanlı bir öneri sistemi ile en çok satan ürünlerin tavsiyesi yapılacaktır. Ardından, matrix factorization yöntemi kullanılarak kişiselleştirilmiş öneriler sunulacaktır. Ayrıca, ürün açıklamaları arasında cosine similarity hesaplanarak, benzer ürünlerin önerilmesi sağlanacaktır. Bu şekilde, şirketin müşteri memnuniyetini ve satış hacmini artırmaya yönelik etkili bir çözüm sunulması amaçlanmaktadır.

### Import Packet

In [5]:
import pandas as pd
import numpy as np

from datetime import date, timedelta
import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

### Import Data

In [7]:
df = pd.read_csv("PBL 5 recommendation data.csv", encoding='latin1')    

In [8]:
df.sample(3)

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,...,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
2240,1879,Betty,Schoenbaum,NaN,1449249407,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CM10,0.0,NaN
3232,2770,Sally,Crank,Company316,1457485504,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BS2,0.0,NaN
680,562,Diane,Gonzalez,NaN,1419781056,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,"23 x 36""",Air Permeable,NaN,NaN,NaN,PF66986,0.0,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194 entries, 0 to 4193
Columns: 181 entries, Customers.id to Products.freight_cost
dtypes: float64(98), int64(10), object(73)
memory usage: 5.8+ MB


In [10]:
df.isnull().sum()

Customers.id                0
Customers.fname             0
Customers.lname             0
Customers.company        3467
Customers.create_date       0
                         ... 
Products.leg_style       4194
Products.seat_size       4032
Products.family_id        325
Products.saved_status     264
Products.freight_cost    4194
Length: 181, dtype: int64

## Popularity Based

Satış hacmine (miktar) ve dolar cinsinden toplam satış tutarına göre en popüler 5 ürünü bulalım.

In [13]:
popular_products = df.groupby('Order_Items.product_id').agg(
    total_qty=('Order_Items.qty', 'sum'),
    total_sales=('Order_Items.price', lambda x: (x * df['Order_Items.qty']).sum())
).reset_index()

top_5 = popular_products.sort_values(by=['total_qty', 'total_sales'], ascending=False).head(5)
top_5

,Order_Items.product_id,total_qty,total_sales
623,1846.0,396,13705.56
686,2107.0,228,12542.26
589,1672.0,220,5559.40
580,1648.0,133,5628.56
622,1842.0,112,1776.10


### Company with a maximum purchase

In [15]:
# Şirketlere göre sipariş sayısı, toplam harcama ve ortalama sipariş tutarını hesaplayalım
top_companies = df.groupby('Orders.company').agg(
    total_orders=('Orders.id', 'count'),            # Toplam sipariş sayısı
    total_spent=('Orders.total', 'sum'),            # Toplam harcama (toplam satış tutarı)
    avg_order_value=('Orders.total', 'mean')        # Ortalama sipariş tutarı
).reset_index()

top_companies_sorted = top_companies.sort_values(by=['total_orders', 'total_spent'], ascending=False).head(5)

print(top_companies_sorted)


    Orders.company  total_orders  total_spent  avg_order_value
53      Company145            27      8530.28       315.936296
37      Company130            17      7330.23       431.190000
378      Company59            15     12790.69       852.712667
371      Company52            13      1364.43       104.956154
82      Company171            12      3297.56       274.796667


### Recommender System Using Matrix Factorization

In [17]:
from sklearn.decomposition import TruncatedSVD

user_product_matrix = df.pivot_table(index=['Customers.id'], columns=['Order_Items.product_id'], values='Order_Items.qty', fill_value=0)

# Perform matrix factorization with SVD
svd = TruncatedSVD(n_components=5)
user_factors = svd.fit_transform(user_product_matrix)
product_factors = svd.components_

user_id = 4
user_index = user_product_matrix.index.get_loc(user_id)
predicted_scores = user_factors[user_index].dot(product_factors)

top_product_indices = predicted_scores.argsort()[::-1][:5]
top_products = user_product_matrix.columns[top_product_indices]
print("Matris çarpanlarına ayırma ile önerilen ürün ID'leri:", top_products.tolist())

Matris çarpanlarına ayırma ile önerilen ürün ID'leri: [4704.0, 15772.0, 1862.0, 11079.0, 4354.0]


In [50]:
def recommend_products(user_id, product_count=5):
  
    try :
        user_index = user_product_matrix.index.get_loc(user_id)
        predicted_scores = user_factors[user_index].dot(product_factors)
        top_product_indices = predicted_scores.argsort()[::-1][:product_count]
        top_products = user_product_matrix.columns[top_product_indices]

        user_name = df[df['Customers.id'] == user_id]['Customers.fname'].values[0]
        last_name = df[df['Customers.id'] == user_id]['Customers.lname'].values[0]
        print('Customer:',user_name ,last_name)

        top_products_name = df[df['Products.id'].isin(top_products)]['Products.name']

        data = pd.DataFrame({'Product ID': top_products, 'Product Name': top_products_name})
    
    except:
        data = 'User not found'

        return data

In [54]:
recommend_products(7, 4)

Customer: peggy  thompson


### Cosine Similarity

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
product_user_matrix = df.pivot_table(index='Order_Items.product_id', columns='Customers.id', values='Order_Items.qty', aggfunc='sum', fill_value=0)

In [23]:
cosine_sim = cosine_similarity(product_user_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index=product_user_matrix.index, columns=product_user_matrix.index)

In [46]:
def similar_products(product_id, top_n=5):
    """
    Recommend similar products based on the product-user matrix using cosine similarity.
    
    Parameters:
    product_id: int, the product id to find similar products
    top_n: int, the number of similar products to return
    
    Returns:
    DataFrame containing Product ID, Product Name, and Shopping Brand of similar products.
    """
    try:
        similar_products = cosine_sim_df[product_id].sort_values(ascending=False)
        similar_products_df = pd.DataFrame(similar_products).reset_index()
        top_n_similar_products = similar_products.index[0:top_n+1]
        product_names = df[df['Order_Items.product_id'].isin(top_n_similar_products)][['Products.name']]
        data = pd.DataFrame({'Product ID': top_n_similar_products, 'Product Name': product_names['Products.name']})
    except KeyError:
        data = 'Product ID not found'
    
    return similar_products_df

In [48]:
similar_products(562,5)

,Order_Items.product_id,562.0
0,562.0,1.0
1,1.0,0.0
2,7538.0,0.0
3,7856.0,0.0
4,7855.0,0.0
...,...,...
1748,1652.0,0.0
1749,1648.0,0.0
1750,1642.0,0.0
1751,1641.0,0.0


Bu proje, bir medikal malzeme şirketinin satışlarını artırmaya yönelik etkili bir öneri sistemi geliştirmeyi amaçlamaktadır. Proje kapsamında gerçekleştirilen analizler, kullanıcı davranışları ve ürün etkileşimleri hakkında değerli bilgiler sağlamıştır. Popülarite tabanlı öneri sistemi ile en çok satan ürünler belirlenmiş ve bu ürünlerin satış hacmi ve gelirleri analiz edilmiştir. Ayrıca, matrix faktörizasyonu yöntemi kullanılarak kişiselleştirilmiş öneriler sunulmuş, bu sayede kullanıcıların ilgi alanlarına uygun ürünler önerilmiştir. Son olarak, ürün açıklamaları arasındaki benzerlikler kullanılarak cosine benzerliği hesaplanmış ve benzer ürünlerin önerilmesi sağlanmıştır.